In [1]:
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq

In [2]:
vqe_handler = VQE(n_qubits=3, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=0, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits)

In [3]:
indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

In [4]:
Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver)
Mcircuit = vqe_handler.give_unitary(Mindexed_circuit, Mresolver)

In [5]:
model = QNN(symbols=list(Mresolver.keys()), observable=vqe_handler.observable)
print(model(circuit), model(Mcircuit))

tf.Tensor([-2.0491378], shape=(1,), dtype=float32) tf.Tensor([-2.9191527], shape=(1,), dtype=float32)


In [6]:
OMenergy, OMresolver, OMhistory = vqe_handler.vqe(Mindexed_circuit, symbols_to_values=Mresolver)
OMcircuit = vqe_handler.give_unitary(Mindexed_circuit, OMresolver)

In [7]:
print(model(Mcircuit), model(OMcircuit))

tf.Tensor([-2.9191527], shape=(1,), dtype=float32) tf.Tensor([-2.999975], shape=(1,), dtype=float32)


In [8]:
Sindexed_circuit, Sresolver, Sindex_to_symbols = simplifier.reduce_circuit(Mindexed_circuit, OMresolver, Mindex_to_symbols)

In [9]:
Scircuit = vqe_handler.give_unitary(Sindexed_circuit,Sresolver)
print(model(Scircuit),OMenergy)

tf.Tensor([-2.9771419], shape=(1,), dtype=float32) tf.Tensor(-2.999976, shape=(), dtype=float32)


In [13]:
OMcircuit

(0, 0): ───Rx(-0.992π)──────────────────────────────@───Rz(0.027π)───Rx(0)────────Rz(-0.036π)───@───Rz(0.027π)───Rx(0)─────────Rz(0.029π)───@───@────────────────
                                                    │                                           │                                           │   │
(0, 1): ───Rx(-0.017π)──────────────────────────────┼───────────────────────────────────────────X───Rx(0.015π)───Rz(-0.034π)───Rx(0.002π)───X───┼────────────────
                                                    │                                                                                           │
(0, 2): ───Rz(-0.032π)───Rx(0.014π)───Rz(-0.023π)───X───Rx(0.026π)───Rz(0.005π)───Rx(-0.029π)───────────────────────────────────────────────────X───Rx(0.981π)───

In [14]:
Scircuit

(0, 0): ───Rx(-0.992π)─────────────────@───Rz(0.027π)───Rx(0)────────Rz(-0.009π)───@───Rx(0.027π)───Rz(0)─────────@───@────────────────
                                       │                                           │                              │   │
(0, 1): ───Rx(-0.002π)─────────────────┼───────────────────────────────────────────X───Rz(0.015π)───Rx(-0.034π)───X───┼────────────────
                                       │                                                                              │
(0, 2): ───Rx(0.014π)────Rz(-0.023π)───X───Rx(0.026π)───Rz(0.005π)───Rx(-0.029π)──────────────────────────────────────X───Rx(0.981π)───

In [19]:
vqe_handler.give_circuit(Sindexed_circuit)[0]

(0, 0): ───Rx(th_0)──────────────@───Rz(th_4)───Rx(th_5)────Rz(th_6)────@───Rx(th_8)────Rz(th_9)────@───@───────────────
                                 │                                      │                           │   │
(0, 1): ───Rx(th_1)──────────────┼──────────────────────────────────────X───Rz(th_10)───Rx(th_11)───X───┼───────────────
                                 │                                                                      │
(0, 2): ───Rx(th_2)───Rz(th_3)───X───Rx(th_7)───Rz(th_12)───Rx(th_13)───────────────────────────────────X───Rx(th_14)───

In [20]:
vqe_handler.give_circuit(Mindexed_circuit)[0]

(0, 0): ───Rx(th_0)─────────────────────────@───Rz(th_5)───Rx(th_6)────Rz(th_7)────@───Rz(th_9)────Rx(th_10)───Rz(th_11)───@───@───────────────
                                            │                                      │                                       │   │
(0, 1): ───Rx(th_1)─────────────────────────┼──────────────────────────────────────X───Rx(th_12)───Rz(th_13)───Rx(th_14)───X───┼───────────────
                                            │                                                                                  │
(0, 2): ───Rz(th_2)───Rx(th_3)───Rz(th_4)───X───Rx(th_8)───Rz(th_15)───Rx(th_16)───────────────────────────────────────────────X───Rx(th_17)───

In [36]:
q=vqe_handler.qubits[0]
u1 = cirq.unitary(cirq.Circuit([cirq.rx(Sresolver["th_2"]).on(q),cirq.rz(Sresolver["th_3"]).on(q)]))
u2 = cirq.unitary(cirq.Circuit([cirq.rz(Mresolver["th_2"]).on(q),cirq.rx("th_3").on(q),cirq.rz("th_4")     ]))

In [ ]:
indexed_circuit = [vqe_handler]

In [ ]:
iid_inserter.resolution_2cnots